### TensorFlow

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model, regularizers
from tensorflow.keras.layers import Conv2D, Dense, Input, Flatten, MaxPooling2D, BatchNormalization, Dropout

from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [3]:
x_train = keras.utils.normalize(x_train, axis = 1)
x_test = keras.utils.normalize(x_test, axis = 1)

In [4]:
model = keras.Sequential()
model.add(Input(shape = (32, 32, 3)))
model.add(Conv2D(32, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, 3, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(Conv2D(128, 3, activation = 'relu'))
model.add(Flatten())
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size = 64, epochs = 15, validation_split = 0.1)

Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.2797 - loss: 1.9720 - val_accuracy: 0.4476 - val_loss: 1.5420
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.4904 - loss: 1.4577 - val_accuracy: 0.5406 - val_loss: 1.3293
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5482 - loss: 1.2986 - val_accuracy: 0.5706 - val_loss: 1.2403
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5901 - loss: 1.1773 - val_accuracy: 0.5840 - val_loss: 1.2162
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6208 - loss: 1.1049 - val_accuracy: 0.6290 - val_loss: 1.0874
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.6415 - loss: 1.0374 - val_accuracy: 0.6402 - val_loss: 1.0657
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.6530 - loss: 1.0016 - val_accuracy: 0.6506 - val_loss: 1.0334
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6694 - loss: 0.9579 - 

In [5]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6526 - loss: 1.0418


[1.0643600225448608, 0.6485999822616577]

### PyTorch

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### 1. Dataloader

In [8]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

train_dataset = torchvision.datasets.CIFAR10(root = '../../data', 
                                           train = True, 
                                           transform = transform,
                                           download = True
                                          )

test_dataset = torchvision.datasets.CIFAR10(root = '../../data', 
                                           train = False, 
                                           transform = transform,
                                           download = True
                                          )

Files already downloaded and verified
Files already downloaded and verified


In [9]:
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = False)

#### 2. Model

In [10]:
class CNN(nn.Module):
    def __init__(self, num_classes = 10):
        super(CNN, self).__init__()
        self.conv_1 = nn.Conv2d(3, 32, 3)
        self.conv_2 = nn.Conv2d(32, 64, 3)
        self.conv_3 = nn.Conv2d(64, 64, 3)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.fc_1 = nn.Linear(64 * 4 * 4, 64)
        self.fc_2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        # input = N, 3, 32, 32
        x = self.conv_1(x)
        x = F.relu(x)
        x = self.max_pool(x)
#         print(x.shape)
        x = self.conv_2(x)
        x = F.relu(x)
        x = self.max_pool(x)
        
        x = self.conv_3(x)
        x = F.relu(x)
#         x = self.max_pool(x)

        x = torch.flatten(x, 1)
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        return x

In [11]:
model = CNN().to(device)

#### 3. Training

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [14]:
n_total_steps = len(train_loader)
for epoch in range(5):
    print('Starting epoch:', epoch)
    running_loss = 0.0
    model.train()
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item()
    print(f'Epoch: {epoch}, loss: {running_loss / n_total_steps}')
print('finished training')

Starting epoch: 0
Epoch: 0, loss: 0.8430596401656353
Starting epoch: 1
Epoch: 1, loss: 0.7773111902858992
Starting epoch: 2
Epoch: 2, loss: 0.7306482018336835
Starting epoch: 3
Epoch: 3, loss: 0.6866341082525802
Starting epoch: 4
Epoch: 4, loss: 0.6453942098700688
finished training


#### 4. Saving Model

In [ ]:
model_save_path = '../../model/cnn_cifar10.pth')
torch.save(model.state_dict(), model_save_path) 

In [ ]:
model = CNN()
model.load_state_dict(torch.load(model_save_path))
model.to(device)

#### 4. Evaluate

In [15]:
model.eval()
with torch.no_grad():
    correct = 0
    samples = len(test_loader.dataset)
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
    print(correct/samples)

0.7169
